# INFO 2950 Project Data Cleaning

#### Import needed packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data Collection and Cleaning

### 1. Load in data

The below data files were found at the following link: https://touringplans.com/blog/2018/06/25/disney-world-wait-times-available-for-data-science-and-machine-learning/

In [2]:
#load metadata csv file
metadata = pd.read_csv('metadata.csv')

#load csv files of each attraction
dwarfs_train = pd.read_csv('7_dwarfs_train.csv')
#alien_saucers = pd.read_csv('alien_saucers.csv')
dinosaur = pd.read_csv('dinosaur.csv')
expedition_everest = pd.read_csv('expedition_everest.csv')
#flight_of_passage = pd.read_csv('flight_of_passage.csv')
kilimanjaro_safaris = pd.read_csv('kilimanjaro_safaris.csv')
#navi_river = pd.read_csv('navi_river.csv')
pirates_of_caribbean = pd.read_csv('pirates_of_caribbean.csv')
rock_n_rollercoaster = pd.read_csv('rock_n_rollercoaster.csv')
#slinky_dog = pd.read_csv('slinky_dog.csv')
soarin = pd.read_csv('soarin.csv')
spaceship_earth = pd.read_csv('spaceship_earth.csv')
splash_mountain = pd.read_csv('splash_mountain.csv')
toy_story_mania = pd.read_csv('toy_story_mania.csv')

### 2. Data Cleaning

Of the attraction wait times data files provided, I chose to keep only those attractions that were around and functioning in 2015, which is when most of the data begins. Therefore, I will not include the following rides in my analysis: **Alien Swarling Saucers, Avatar Flight of Passage, Na'vi River Journey, and Slinky Dog Dash**. 

In [3]:
dwarfs_train.head()

,date,datetime,SACTMIN,SPOSTMIN
0,01/01/2015,2015-01-01 07:51:12,NaN,45.0
1,01/01/2015,2015-01-01 08:02:13,NaN,60.0
2,01/01/2015,2015-01-01 08:05:30,54.0,NaN
3,01/01/2015,2015-01-01 08:09:12,NaN,60.0
4,01/01/2015,2015-01-01 08:16:12,NaN,60.0


When going through the datasets for each attraction, I noticed they have different reporting times and number of reports per day. Therefore, in order to merge the attraction wait times into one data frame, I will get an average hourly wait time for each hour that the park is open. I will only look at normal park hours; I am choosing to ignore wait time data for Extra Magic Hours, which are hours the parks are open only for specific guests. To further simplify, I will ignore any hours after midnight that the park is open. For example, on certain holidays, the parks are open past midnnight. This will complicate date analysis, so I am choosing to ignore this data. 

In addition, to simplify analyses, I am ony going to look at `SPOSTMIN`, the posted wait time in minutes. I am going to ignore, and get rid of, `SACTMIN`, the actual wait time in minutes, because besides a few occurrences, this column is empty. 

**Step 1:** 
 - Create new attraction datasets with only `datetime` and `SPOSTMIN` columns. 
 - Get rid of NaN values in `SPOSTMIN` columns (only occurs when `SACTMIN` is reported instead). There are enough wait times to analyze already, so I do not consider this a big deal. 

In [4]:
dwarfs_train_clean = dwarfs_train[['datetime', 'SPOSTMIN']].dropna().copy()
dinosaur_clean = dinosaur[['datetime', 'SPOSTMIN']].dropna().copy()
expedition_everest_clean = expedition_everest[['datetime', 'SPOSTMIN']].dropna().copy()
kilimanjaro_safaris_clean = kilimanjaro_safaris[['datetime', 'SPOSTMIN']].dropna().copy()
pirates_of_caribbean_clean = pirates_of_caribbean[['datetime', 'SPOSTMIN']].dropna().copy()
rock_n_rollercoaster_clean = rock_n_rollercoaster[['datetime', 'SPOSTMIN']].dropna().copy()
soarin_clean = soarin[['datetime', 'SPOSTMIN']].dropna().copy()
spaceship_earth_clean = spaceship_earth[['datetime', 'SPOSTMIN']].dropna().copy()
splash_mountain_clean = splash_mountain[['datetime', 'SPOSTMIN']].dropna().copy()
toy_story_mania_clean = toy_story_mania[['datetime', 'SPOSTMIN']].dropna().copy()

**Step 2:**
 - Convert `datetime` column from type string to datetime
 - From this datetime object, split into two new columns `date` and `hour_of_day`. This will make it easier to get the average wait time each hour of every day. 

In [5]:
#Function to change datetime column to type datetime
def change_to_datetime(ride):
    '''
    param ride: the Dataframe to change the datetime column to type datetime
    '''
    ride.loc[:, 'datetime'] = pd.to_datetime(ride.datetime, format="%Y-%m-%d %H:%M:%S")
    ride['date'] = ride.datetime.dt.date
    ride['hour_of_day'] = ride.datetime.dt.hour

In [6]:
#Create new list of all clean data for each ride
all_rides_clean = [dwarfs_train_clean, dinosaur_clean, expedition_everest_clean, kilimanjaro_safaris_clean,
        pirates_of_caribbean_clean, rock_n_rollercoaster_clean, soarin_clean, spaceship_earth_clean,
        splash_mountain_clean, toy_story_mania_clean]

#Call above function to change datetime column to type datetime
for ride in all_rides_clean:
    change_to_datetime(ride)

In [7]:
dwarfs_train_clean.head()

,datetime,SPOSTMIN,date,hour_of_day
0,2015-01-01 07:51:12,45.0,2015-01-01,7
1,2015-01-01 08:02:13,60.0,2015-01-01,8
3,2015-01-01 08:09:12,60.0,2015-01-01,8
4,2015-01-01 08:16:12,60.0,2015-01-01,8
6,2015-01-01 08:23:12,60.0,2015-01-01,8


**Step 3:**
 - Reorganize dataframes for each attraction so the columns are in order: `date`, `hour_of_day`, and `SPOSTMIN`
 - Rename the `SPOSTMIN` column to `wait_time` to make the column name more explanatory

In [8]:
#Reorganize dataframes to date, hour, wait_time
dwarfs_train_hourly = dwarfs_train_clean[['date', 'hour_of_day', 'SPOSTMIN']].copy()
dinosaur_hourly = dinosaur_clean[['date', 'hour_of_day', 'SPOSTMIN']].copy()
expedition_everest_hourly = expedition_everest_clean[['date', 'hour_of_day', 'SPOSTMIN']].copy()
kilimanjaro_safaris_hourly = kilimanjaro_safaris_clean[['date', 'hour_of_day', 'SPOSTMIN']].copy()
pirates_of_caribbean_hourly = pirates_of_caribbean_clean[['date', 'hour_of_day', 'SPOSTMIN']].copy()
rock_n_rollercoaster_hourly = rock_n_rollercoaster_clean[['date', 'hour_of_day', 'SPOSTMIN']].copy()
soarin_hourly = soarin_clean[['date', 'hour_of_day', 'SPOSTMIN']].copy()
spaceship_earth_hourly = spaceship_earth_clean[['date', 'hour_of_day', 'SPOSTMIN']].copy()
splash_mountain_hourly = splash_mountain_clean[['date', 'hour_of_day', 'SPOSTMIN']].copy()
toy_story_mania_hourly = toy_story_mania_clean[['date', 'hour_of_day', 'SPOSTMIN']].copy()

In [9]:
def rename_columns(ride):
    ride.columns = ['date', 'hour_of_day', 'wait_time']

In [10]:
all_rides_hourly = [dwarfs_train_hourly, dinosaur_hourly, expedition_everest_hourly, kilimanjaro_safaris_hourly,
        pirates_of_caribbean_hourly, rock_n_rollercoaster_hourly, soarin_hourly, spaceship_earth_hourly,
        splash_mountain_hourly, toy_story_mania_hourly]

for ride in all_rides_hourly:
    rename_columns(ride)

**Step 4:** 
 - When an attraction was closed on certain days/times, `wait_time = -999.0`. This might skew analyses so I will get rid of any wait times less than zero.  

In [11]:
dwarfs_train_hourly2 = dwarfs_train_hourly[dwarfs_train_hourly.wait_time >=0]
dinosaur_hourly2 = dinosaur_hourly[dinosaur_hourly.wait_time >=0]
expedition_everest_hourly2 = expedition_everest_hourly[expedition_everest_hourly.wait_time >=0]
kilimanjaro_safaris_hourly2 = kilimanjaro_safaris_hourly[kilimanjaro_safaris_hourly.wait_time >=0]
pirates_of_caribbean_hourly2 = pirates_of_caribbean_hourly[pirates_of_caribbean_hourly.wait_time >=0]
rock_n_rollercoaster_hourly2 = rock_n_rollercoaster_hourly[rock_n_rollercoaster_hourly.wait_time >=0]
soarin_hourly2 = soarin_hourly[soarin_hourly.wait_time >=0]
spaceship_earth_hourly2 = spaceship_earth_hourly[spaceship_earth_hourly.wait_time >=0]
splash_mountain_hourly2 = splash_mountain_hourly[splash_mountain_hourly.wait_time >=0]
toy_story_mania_hourly2 = toy_story_mania_hourly[toy_story_mania_hourly.wait_time >=0]

**Step 5:**
 - Get the average wait time for each hour of every day using the `groupby()` function
 - Use `reset_index()` to convert this data back into a dataframe 

In [12]:
dwarfs_train_hourly_avg = dwarfs_train_hourly2.groupby(['date', 'hour_of_day']).mean().round(decimals=2)
dwarfs_train_hourly_avg = pd.DataFrame(dwarfs_train_hourly_avg.reset_index())

dinosaur_hourly_avg = dinosaur_hourly2.groupby(['date', 'hour_of_day']).mean().round(decimals=2)
dinosaur_hourly_avg = pd.DataFrame(dinosaur_hourly_avg.reset_index())

expedition_everest_hourly_avg = expedition_everest_hourly2.groupby(['date', 'hour_of_day']).mean().round(decimals=2)
expedition_everest_hourly_avg = pd.DataFrame(expedition_everest_hourly_avg.reset_index())

kilimanjaro_safaris_hourly_avg = kilimanjaro_safaris_hourly2.groupby(['date', 'hour_of_day']).mean().round(decimals=2)
kilimanjaro_safaris_hourly_avg = pd.DataFrame(kilimanjaro_safaris_hourly_avg.reset_index())

pirates_of_caribbean_hourly_avg = pirates_of_caribbean_hourly2.groupby(['date', 'hour_of_day']).mean().round(decimals=2)
pirates_of_caribbean_hourly_avg = pd.DataFrame(pirates_of_caribbean_hourly_avg.reset_index())

rock_n_rollercoaster_hourly_avg = rock_n_rollercoaster_hourly2.groupby(['date', 'hour_of_day']).mean().round(decimals=2)
rock_n_rollercoaster_hourly_avg = pd.DataFrame(rock_n_rollercoaster_hourly_avg.reset_index())

soarin_hourly_avg = soarin_hourly2.groupby(['date', 'hour_of_day']).mean().round(decimals=2)
soarin_hourly_avg = pd.DataFrame(soarin_hourly_avg.reset_index())

spaceship_earth_hourly_avg = spaceship_earth_hourly2.groupby(['date', 'hour_of_day']).mean().round(decimals=2)
spaceship_earth_hourly_avg = pd.DataFrame(spaceship_earth_hourly_avg.reset_index())

splash_mountain_hourly_avg = splash_mountain_hourly2.groupby(['date', 'hour_of_day']).mean().round(decimals=2)
splash_mountain_hourly_avg = pd.DataFrame(splash_mountain_hourly_avg.reset_index())

toy_story_mania_hourly_avg = toy_story_mania_hourly2.groupby(['date', 'hour_of_day']).mean().round(decimals=2)
toy_story_mania_hourly_avg = pd.DataFrame(toy_story_mania_hourly_avg.reset_index())

**Step 6:**
 - Add the initials of each attraction to the `wait_time` column name. This will distinguish each attraction when we merge the wait times together.

In [13]:
#Change wait_time column name to include attraction initials
dwarfs_train_hourly_avg.columns = ['date', 'hour_of_day', 'DT_wait_time']
dinosaur_hourly_avg.columns = ['date', 'hour_of_day', 'D_wait_time']
expedition_everest_hourly_avg.columns = ['date', 'hour_of_day', 'EE_wait_time']
kilimanjaro_safaris_hourly_avg.columns = ['date', 'hour_of_day', 'KS_wait_time']
pirates_of_caribbean_hourly_avg.columns = ['date', 'hour_of_day', 'POC_wait_time']
rock_n_rollercoaster_hourly_avg.columns = ['date', 'hour_of_day', 'RNR_wait_time']
soarin_hourly_avg.columns = ['date', 'hour_of_day', 'S_wait_time']
spaceship_earth_hourly_avg.columns = ['date', 'hour_of_day', 'SE_wait_time']
splash_mountain_hourly_avg.columns = ['date', 'hour_of_day', 'SM_wait_time']
toy_story_mania_hourly_avg.columns = ['date', 'hour_of_day', 'TSM_wait_time']

In [14]:
dwarfs_train_hourly_avg.head()

,date,hour_of_day,DT_wait_time
0,2015-01-01,7,45.00
1,2015-01-01,8,78.33
2,2015-01-01,9,103.33
3,2015-01-01,10,87.50
4,2015-01-01,11,85.56


Now we only want the hourly wait times for when the park was open for normal hours. Also, to make analyses easier, I am ignoring the hours past midnight so I can easily distinguish between different days. In order to do this, we need to use data from `metadata`.

**Step 7:**
 - Keep only the columns from `metadata` that I am interested in and might use in anlayses. These include `DATE`, `DAYOFWEEK`, `DAYOFYEAR`, `WEEKOFYEAR`, `MONTHOFYEAR`, `SEASON`, `HOLIDAYN`, `HOLIDAY`, `WDWMEANTEMP`, `INSESSION`, `MKOPEN`, `MKCLOSE`, `EPOPEN`, `EOPCLOSE`, `HSOPEN`, `HSCLOSE`, `AKOPEN`, `AKCLOSE`, `WEATHER_WDWPRECIP`

In [15]:
metadata_new = metadata[['DATE', 'DAYOFWEEK', 'DAYOFYEAR', 'WEEKOFYEAR', 'MONTHOFYEAR', 'SEASON', 'HOLIDAYN', 'HOLIDAY', 
                        'WDWMEANTEMP', 'inSession', 'MKOPEN', 'MKCLOSE', 'EPOPEN', 'EPCLOSE', 'HSOPEN', 'HSCLOSE', 'AKOPEN', 
                        'AKCLOSE', 'WEATHER_WDWPRECIP']].copy()

metadata_new

,DATE,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR,SEASON,HOLIDAYN,HOLIDAY,WDWMEANTEMP,inSession,MKOPEN,MKCLOSE,EPOPEN,EPCLOSE,HSOPEN,HSCLOSE,AKOPEN,AKCLOSE,WEATHER_WDWPRECIP
0,01/01/2015,5,0,0,1,CHRISTMAS PEAK,nyd,1,66.41,0%,8:00,25:00,8:00,21:00,8:00,22:00,8:00,19:00,0.12
1,01/02/2015,6,1,0,1,CHRISTMAS,NaN,0,69.36,0%,8:00,25:00,8:00,21:00,8:00,22:00,8:00,20:00,0.12
2,01/03/2015,7,2,0,1,CHRISTMAS,NaN,0,75.22,0%,9:00,25:00,9:00,21:00,9:00,22:00,8:00,19:00,0.07
3,01/04/2015,1,3,1,1,CHRISTMAS,NaN,0,75.95,67%,9:00,24:00,9:00,21:00,9:00,20:00,9:00,20:00,0.12
4,01/05/2015,2,4,1,1,CHRISTMAS,NaN,0,64.60,67%,9:00,23:00,9:00,21:00,9:00,20:00,9:00,17:00,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2127,12/26/2019,5,359,51,12,CHRISTMAS,NaN,0,63.53,0%,8:00,24:00,9:00,21:30,6:00,22:00,8:00,21:00,0.07
2128,12/27/2019,6,360,51,12,CHRISTMAS PEAK,NaN,0,54.72,0%,8:00,24:00,9:00,21:30,6:00,22:00,8:00,22:00,0.08
2129,12/28/2019,7,361,51,12,CHRISTMAS PEAK,NaN,0,56.74,0%,8:00,24:00,9:00,21:30,6:00,22:00,8:00,22:00,0.08
2130,12/29/2019,1,362,52,12,CHRISTMAS PEAK,NaN,0,56.74,0%,8:00,24:00,9:00,21:30,6:00,22:00,8:00,23:00,0.08


**Step 8:**
 - Change the type of the `DATE` column from string to a datetime object. Extract date part of it to compare with `date` column in attraction dataframes. 

In [16]:
#Change date to date object
metadata_new.loc[:, 'DATE'] = pd.to_datetime(metadata_new.DATE, format="%m/%d/%Y")
metadata_new['DATE'] = metadata_new.DATE.dt.date

metadata_new

,DATE,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR,SEASON,HOLIDAYN,HOLIDAY,WDWMEANTEMP,inSession,MKOPEN,MKCLOSE,EPOPEN,EPCLOSE,HSOPEN,HSCLOSE,AKOPEN,AKCLOSE,WEATHER_WDWPRECIP
0,2015-01-01,5,0,0,1,CHRISTMAS PEAK,nyd,1,66.41,0%,8:00,25:00,8:00,21:00,8:00,22:00,8:00,19:00,0.12
1,2015-01-02,6,1,0,1,CHRISTMAS,NaN,0,69.36,0%,8:00,25:00,8:00,21:00,8:00,22:00,8:00,20:00,0.12
2,2015-01-03,7,2,0,1,CHRISTMAS,NaN,0,75.22,0%,9:00,25:00,9:00,21:00,9:00,22:00,8:00,19:00,0.07
3,2015-01-04,1,3,1,1,CHRISTMAS,NaN,0,75.95,67%,9:00,24:00,9:00,21:00,9:00,20:00,9:00,20:00,0.12
4,2015-01-05,2,4,1,1,CHRISTMAS,NaN,0,64.60,67%,9:00,23:00,9:00,21:00,9:00,20:00,9:00,17:00,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2127,2019-12-26,5,359,51,12,CHRISTMAS,NaN,0,63.53,0%,8:00,24:00,9:00,21:30,6:00,22:00,8:00,21:00,0.07
2128,2019-12-27,6,360,51,12,CHRISTMAS PEAK,NaN,0,54.72,0%,8:00,24:00,9:00,21:30,6:00,22:00,8:00,22:00,0.08
2129,2019-12-28,7,361,51,12,CHRISTMAS PEAK,NaN,0,56.74,0%,8:00,24:00,9:00,21:30,6:00,22:00,8:00,22:00,0.08
2130,2019-12-29,1,362,52,12,CHRISTMAS PEAK,NaN,0,56.74,0%,8:00,24:00,9:00,21:30,6:00,22:00,8:00,23:00,0.08


**Step 9:**
 - Convert the open and closing time columns (`MKOPEN`, `MKCLOSE`, `EPOPEN`, `EPCLOSE`, `HSOPEN`, `HSCLOSE`, `AKOPEN`, `AKCLOSE`) from type string to a datetime object and then extract the hour. 
 - Before doing so, I must address the times that are not in correct military time format. This occurs when the park is open past midnight. I chose to change all of these times to 23:00. 

In [17]:
def change_to_time(time_col):
    '''
    param time_col: a Pandas Series consisting of strings that need to be changed to time objects and cut to just the hour
    '''
    ## get rows where time='24:00' or '25:00' or '26:00' or '27:00' or '29:59'
    too_late_times1 = metadata_new.loc[:, time_col] == '24:00'
    too_late_times2 = metadata_new.loc[:, time_col] == '25:00'
    too_late_times3 = metadata_new.loc[:, time_col] == '26:00'
    too_late_times4 = metadata_new.loc[:, time_col] == '27:00'
    too_late_times5 = metadata_new.loc[:, time_col] == '29:59'

    ## locate rows with too late times and replace with '23:00'
    metadata_new.loc[ too_late_times1 , time_col ] = '23:00'
    metadata_new.loc[ too_late_times2 , time_col ] = '23:00'
    metadata_new.loc[ too_late_times3 , time_col ] = '23:00'
    metadata_new.loc[ too_late_times4 , time_col ] = '23:00'
    metadata_new.loc[ too_late_times5 , time_col ] = '23:00'
    
    metadata_new.loc[:, time_col] = pd.to_datetime(metadata_new[time_col], format="%H:%M")
    metadata_new[time_col] = metadata_new[time_col].dt.hour

In [18]:
time_columns = ['MKOPEN', 'MKCLOSE', 'EPOPEN', 'EPCLOSE', 'HSOPEN', 'HSCLOSE', 'AKOPEN', 'AKCLOSE']

for col in time_columns:
    change_to_time(col)

**Step 10:**
 - Convert `inSession` column from type String to a float

In [19]:
metadata_new['inSession'] = metadata_new.inSession.str.slice(stop=-1).astype(float)/100
metadata_new

,DATE,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR,SEASON,HOLIDAYN,HOLIDAY,WDWMEANTEMP,inSession,MKOPEN,MKCLOSE,EPOPEN,EPCLOSE,HSOPEN,HSCLOSE,AKOPEN,AKCLOSE,WEATHER_WDWPRECIP
0,2015-01-01,5,0,0,1,CHRISTMAS PEAK,nyd,1,66.41,0.00,8,23,8,21,8,22,8,19,0.12
1,2015-01-02,6,1,0,1,CHRISTMAS,NaN,0,69.36,0.00,8,23,8,21,8,22,8,20,0.12
2,2015-01-03,7,2,0,1,CHRISTMAS,NaN,0,75.22,0.00,9,23,9,21,9,22,8,19,0.07
3,2015-01-04,1,3,1,1,CHRISTMAS,NaN,0,75.95,0.67,9,23,9,21,9,20,9,20,0.12
4,2015-01-05,2,4,1,1,CHRISTMAS,NaN,0,64.60,0.67,9,23,9,21,9,20,9,17,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2127,2019-12-26,5,359,51,12,CHRISTMAS,NaN,0,63.53,0.00,8,23,9,21,6,22,8,21,0.07
2128,2019-12-27,6,360,51,12,CHRISTMAS PEAK,NaN,0,54.72,0.00,8,23,9,21,6,22,8,22,0.08
2129,2019-12-28,7,361,51,12,CHRISTMAS PEAK,NaN,0,56.74,0.00,8,23,9,21,6,22,8,22,0.08
2130,2019-12-29,1,362,52,12,CHRISTMAS PEAK,NaN,0,56.74,0.00,8,23,9,21,6,22,8,23,0.08


**Step 11:**
- For some reason there's a bunch of duplicates of dates from 12/29/2017 to 1/6/2018, so I want to get rid of those. 

In [20]:
#Get rid of duplicates
metadata_no_dups = metadata_new.drop_duplicates()
metadata_no_dups

,DATE,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR,SEASON,HOLIDAYN,HOLIDAY,WDWMEANTEMP,inSession,MKOPEN,MKCLOSE,EPOPEN,EPCLOSE,HSOPEN,HSCLOSE,AKOPEN,AKCLOSE,WEATHER_WDWPRECIP
0,2015-01-01,5,0,0,1,CHRISTMAS PEAK,nyd,1,66.41,0.00,8,23,8,21,8,22,8,19,0.12
1,2015-01-02,6,1,0,1,CHRISTMAS,NaN,0,69.36,0.00,8,23,8,21,8,22,8,20,0.12
2,2015-01-03,7,2,0,1,CHRISTMAS,NaN,0,75.22,0.00,9,23,9,21,9,22,8,19,0.07
3,2015-01-04,1,3,1,1,CHRISTMAS,NaN,0,75.95,0.67,9,23,9,21,9,20,9,20,0.12
4,2015-01-05,2,4,1,1,CHRISTMAS,NaN,0,64.60,0.67,9,23,9,21,9,20,9,17,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2127,2019-12-26,5,359,51,12,CHRISTMAS,NaN,0,63.53,0.00,8,23,9,21,6,22,8,21,0.07
2128,2019-12-27,6,360,51,12,CHRISTMAS PEAK,NaN,0,54.72,0.00,8,23,9,21,6,22,8,22,0.08
2129,2019-12-28,7,361,51,12,CHRISTMAS PEAK,NaN,0,56.74,0.00,8,23,9,21,6,22,8,22,0.08
2130,2019-12-29,1,362,52,12,CHRISTMAS PEAK,NaN,0,56.74,0.00,8,23,9,21,6,22,8,23,0.08


**Step 12:**
 - Group attractions by park and merge these dataframes on `date` and `hour_of_day` columns. I use an outer join to make sure I don't get rid of any data. This creates some NaN values when some attractions are missing wait times for certain hours or was closed. 

In [21]:
#Combine ride datasets into dataset for each park
#OUTER join on data and hour_of_day column

#Magic Kingdom: Dwarfs Train, Pirates of Caribbean, Splash Mountain
mk_data = pd.merge(dwarfs_train_hourly_avg, pirates_of_caribbean_hourly_avg, how='outer', on=['date', 'hour_of_day'])
mk_data = pd.merge(mk_data, splash_mountain_hourly_avg, how='outer', on=['date', 'hour_of_day'])

#Epcot: Soarin', Spaceship Earth
ep_data = pd.merge(soarin_hourly_avg, spaceship_earth_hourly_avg, how='outer', on=['date', 'hour_of_day'])

#Hollywood Studios: Rock 'N Rollercoaster, Toy Story Mania
hs_data = pd.merge(rock_n_rollercoaster_hourly_avg, toy_story_mania_hourly_avg, how='outer', on=['date', 'hour_of_day'])

#Animal Kingdom: Dinosaur, Expwedition Everest, Kilimanjaro Safaris
ak_data = pd.merge(dinosaur_hourly_avg, expedition_everest_hourly_avg, how='outer', on=['date', 'hour_of_day'])
ak_data = pd.merge(ak_data, kilimanjaro_safaris_hourly_avg, how='outer', on=['date', 'hour_of_day'])

In [22]:
mk_data.head()

,date,hour_of_day,DT_wait_time,POC_wait_time,SM_wait_time
0,2015-01-01,7,45.00,NaN,5.00
1,2015-01-01,8,78.33,NaN,5.00
2,2015-01-01,9,103.33,NaN,5.00
3,2015-01-01,10,87.50,27.5,15.56
4,2015-01-01,11,85.56,62.5,21.88


**Step 13:**
 - Rename the `DATE` column in `metadata` to `date` in order to merge with attraction dataframes
 - Merge each park dataframe with the corresponding open and closing time columns from `metadata`
 - Get rid of hours from each park dataframe where the park was not open

In [23]:
#Rename for date columns to match
metadata_no_dups = metadata_no_dups.rename(columns={'DATE':'date'})

#Add opening and closing hours for each park
mk_data_merged = pd.merge(mk_data, metadata_no_dups[['date','MKOPEN','MKCLOSE']], how='outer', on='date')

ep_data_merged = pd.merge(ep_data, metadata_no_dups[['date','EPOPEN','EPCLOSE']], how='outer', on='date')

hs_data_merged = pd.merge(hs_data, metadata_no_dups[['date','HSOPEN','HSCLOSE']], how='outer', on='date')

ak_data_merged = pd.merge(ak_data, metadata_no_dups[['date','AKOPEN','AKCLOSE']], how='outer', on='date')

#Only keep hours >= Open or <= Close
mk_after_open = mk_data_merged[mk_data_merged.hour_of_day >= mk_data_merged.MKOPEN]
mk_open = mk_after_open[mk_after_open.hour_of_day < mk_after_open.MKCLOSE]

ep_after_open = ep_data_merged[ep_data_merged.hour_of_day >= ep_data_merged.EPOPEN]
ep_open = ep_after_open[ep_after_open.hour_of_day < ep_after_open.EPCLOSE]

hs_after_open = hs_data_merged[hs_data_merged.hour_of_day >= hs_data_merged.HSOPEN]
hs_open = hs_after_open[hs_after_open.hour_of_day < hs_after_open.HSCLOSE]

ak_after_open = ak_data_merged[ak_data_merged.hour_of_day >= ak_data_merged.AKOPEN]
ak_open = ak_after_open[ak_after_open.hour_of_day < ak_after_open.AKCLOSE]

In [24]:
mk_open.head()

,date,hour_of_day,DT_wait_time,POC_wait_time,SM_wait_time,MKOPEN,MKCLOSE
1,2015-01-01,8.0,78.33,NaN,5.00,8.0,23.0
2,2015-01-01,9.0,103.33,NaN,5.00,8.0,23.0
3,2015-01-01,10.0,87.50,27.5,15.56,8.0,23.0
4,2015-01-01,11.0,85.56,62.5,21.88,8.0,23.0
5,2015-01-01,12.0,121.11,50.0,28.12,8.0,23.0


**Step 14:**
 - Reorder data by date and then by hour for each park
 - Get rid of open and close time columns for each park

In [25]:
#Put data back in date order
mk_sorted = mk_open.sort_values(by=['date', 'hour_of_day'])
ep_sorted = ep_open.sort_values(by=['date', 'hour_of_day'])
hs_sorted = hs_open.sort_values(by=['date', 'hour_of_day'])
ak_sorted = ak_open.sort_values(by=['date', 'hour_of_day'])

In [26]:
#Get rid of OPEN and CLOSE times in all data sets
metadata_final = metadata_no_dups.drop(['MKOPEN', 'MKCLOSE', 'EPOPEN', 
                                       'EPCLOSE', 'HSOPEN', 'HSCLOSE', 'AKOPEN', 'AKCLOSE'], axis=1)
mk_final = mk_sorted.drop(['MKOPEN', 'MKCLOSE'], axis=1)
ep_final = ep_sorted.drop(['EPOPEN', 'EPCLOSE'], axis=1)
hs_final = hs_sorted.drop(['HSOPEN', 'HSCLOSE'], axis=1)
ak_final = ak_sorted.drop(['AKOPEN', 'AKCLOSE'], axis=1)

In [33]:
ak_final

,date,hour_of_day,D_wait_time,EE_wait_time,KS_wait_time
1,2015-01-01,8.0,9.38,4.50,8.75
2,2015-01-01,9.0,10.00,11.67,5.00
3,2015-01-01,10.0,18.75,31.25,6.25
4,2015-01-01,11.0,20.00,36.88,16.25
5,2015-01-01,12.0,26.25,50.00,35.56
...,...,...,...,...,...
22358,2019-12-30,18.0,60.00,78.33,58.33
22359,2019-12-30,19.0,68.12,68.12,NaN
22360,2019-12-30,20.0,41.88,57.50,NaN
22361,2019-12-30,21.0,18.12,35.62,NaN


In [28]:
metadata_final

,date,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR,SEASON,HOLIDAYN,HOLIDAY,WDWMEANTEMP,inSession,WEATHER_WDWPRECIP
0,2015-01-01,5,0,0,1,CHRISTMAS PEAK,nyd,1,66.41,0.00,0.12
1,2015-01-02,6,1,0,1,CHRISTMAS,NaN,0,69.36,0.00,0.12
2,2015-01-03,7,2,0,1,CHRISTMAS,NaN,0,75.22,0.00,0.07
3,2015-01-04,1,3,1,1,CHRISTMAS,NaN,0,75.95,0.67,0.12
4,2015-01-05,2,4,1,1,CHRISTMAS,NaN,0,64.60,0.67,0.12
...,...,...,...,...,...,...,...,...,...,...,...
2127,2019-12-26,5,359,51,12,CHRISTMAS,NaN,0,63.53,0.00,0.07
2128,2019-12-27,6,360,51,12,CHRISTMAS PEAK,NaN,0,54.72,0.00,0.08
2129,2019-12-28,7,361,51,12,CHRISTMAS PEAK,NaN,0,56.74,0.00,0.08
2130,2019-12-29,1,362,52,12,CHRISTMAS PEAK,NaN,0,56.74,0.00,0.08


### 3. Export dataframes to CSV files

In [36]:
metadata_final.to_csv("metadata_final.csv", index=False)
mk_final.to_csv("mk_final.csv", index=False)
ep_final.to_csv("ep_final.csv", index=False)
hs_final.to_csv("hs_final.csv", index=False)
ak_final.to_csv("ak_final.csv", index=False)